## Прогнозирование стоимости автомобиля по характеристикам

# Описание признаков в тестовом датасете:
* bodyType - тип кузова
* brand - марка автомобиля 
* car_url - адрес страницы с объявлением 
* color - цвет автомобиля
* complectation_dict - словарь с параметрами комплектации авто
* description - описание автомобиля 
* engineDisplacement - объем двигателя
* enginePower - мощность двигателя
* equipment_dict - словарь оснащения авто 
* fuelType - тип топлива 
* image - изображение 
* mileage - пробег 
* modelDate - дата выпуска модели
* model_info - информация о модели
* model_name - название модели
* name - имя
* numberOfDoors - кол-во дверей
* parsing_unixtime - когда был произведен парсинг данных
* priceCurrency - валюта
* productionDate - год производства
* sell_id
* super_gen - словарь с дополинтельной информацией
* vehicleConfiguration - конфигурация транспортного средства
* vehicleTransmission - трансмиссия
* vendor - страна-изготовитель
* Владельцы - кол-во владельцев
* Владение - период владения
* ПТС
* Привод
* Руль
* Состояние
* Таможня

In [ ]:
import numpy as np 
import pandas as pd 
import sys
import time
import datetime
import requests as r
import json
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import pandas_profiling
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor  
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import clone

In [ ]:
# фиксируем RANDOM_SEED
RANDOM_SEED = 42

In [ ]:
#запишем функцию для вычисления итоговой метрики
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Код парсинга

In [ ]:
'''START_URL='https://auto.ru/moskva/cars/used/'

pages_url_list=[]'''

In [ ]:
'''while len(pages_url_list)<5000: 
    for i in range(1,99):   
        response = r.get( START_URL+'?page=%s'%i)
        page = BeautifulSoup(response.text, 'html.parser')
        for link in page.findAll('a',{"class": "Link OfferThumb"}):
            try:
                if link['href'] not in pages_url_list: 
                    pages_url_list.append(link['href'])
            except:
                pass
    print(len(pages_url_list))   
    time.sleep(10)'''

In [ ]:
"""cars_list = []
for item in pages_url_list:
    response = r.get(item)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    if page.find('div', class_='CardSold') == None:
        try:
            json_data = json.loads(
                page.find('script', type="application/ld+json").string)
        except:
            print(item)
            pass
        cls_str = 'CardInfoRow_'
        span_str = 'CardInfoRow__cell'
        try:
            cars_list.append({
            'bodyType': json_data['bodyType'],
            'brand': json_data['brand'],
            'car_url': json_data['offers']['url'],
            'color': json_data['color'],
            'description': json_data['description'],
            'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
            'enginePower': json_data['vehicleEngine']['enginePower'],
            'fuelType': json_data['fuelType'],
            'image': json_data['image'],
            'mileage': page.find(
                'li', class_=cls_str+'kmAge').find_all('span')[1].text,
            'modelDate': json_data['modelDate'],
            'model_name': json_data['name'],
            'name': json_data['vehicleEngine']['name'],
            'numberOfDoors': json_data['numberOfDoors'],
            'parsing_unixtime': int(time.time()),
            'priceCurrency': json_data['offers']['priceCurrency'],
            'productionDate': json_data['productionDate'],
            'sell_id': page.find(
                'div', title='Идентификатор объявления').text,
            'vehicleConfiguration': json_data['vehicleConfiguration'],
            'vehicleTransmission': json_data['vehicleTransmission'],
            'Владельцы': page.find(
                'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
            'ПТС': page.find(
                'li', class_=cls_str+'pts').find_all('span')[1].text,
            'Привод': page.find(
                'li', class_=cls_str+'drive').find_all('span')[1].text,
            'Руль': page.find(
                'li', class_=cls_str+'wheel').find_all('span')[1].text,
            'Состояние': page.find(
                'li', class_=cls_str+'state').find_all('span')[1].text,
            'Таможня': page.find(
                'li', class_=cls_str+'customs').find_all('span')[1].text,
            'price': page.find(
                'span', class_='OfferPriceCaption__price').text
            })
        except:
            print(item)
            pass

   # else cars_list = []

    #print('Ожидаю 0.1 секунды...')
    time.sleep(0.1)"""

In [ ]:
"""# переводим в удобный для нас вид
df = pd.DataFrame(cars_list)
# проверяем на дубликаты, можно по двум признакам car_url или sell_id
df['sell_id'].duplicated().sum()
# сохраняем в csv
df.to_csv('cars.csv', index=False)"""

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN  = '../input/cars-df/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'cars.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.sample(5)

In [ ]:
train.info()
train.isna().sum()

In [ ]:
test.sample(5)

In [ ]:
test.info()
test.isna().sum()

In [ ]:
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', train.shape)
display(train.head(5))
print('Размерность тестового датасета: ', test.shape)
display(test.head(5))

## Data Preprocessing

### удаляем дубликаты

In [ ]:
train = train.drop_duplicates(subset=['car_url'])

### проверим датасеты на дубликаты по уникальному идентификатору объявления

In [ ]:
train['car_url'].duplicated().sum()
test['sell_id'].duplicated().sum()

### удалим из тестового датафрейма данные, которые не удалось спарсить в тренировочный датасет

In [ ]:
test.drop(['complectation_dict', 'equipment_dict', 'model_info',
           'super_gen', 'vendor', 'Владение'], axis=1, inplace=True)

In [ ]:
test['price'] = 0
df_train = train
df_test = test

# удалим строки, где отсутствует цена
df_train = df_train.dropna(axis=0, subset=['price'])

# объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.sample(5)

### еще раз проверим на дубликаты

In [ ]:
data['car_url'].value_counts()
data = data.drop_duplicates(subset=['car_url'])

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.isna().sum()

Краткий итог: в тренировочной выборке 33531 объявлений, в тестовой - 34686. Всего данных после удаления дубликатов - 65609 объявлений. Имеется 28 признаков. Из числовых признаков у нас: modelDate, numberOfDoors, productionDate,parsing_unixtime. Остальные - типа Object. Пропуски в признаках price,parsing_unixtime,sell_id появились исскуственно. Также добавился признак sample для обозначения принадлежности данных и последующего разделения.

# Рассмотрим каждый признак

In [ ]:
#числовые:
num_cols = []

# бинарные, количество уникальных значений которых равно двум
bin_cols = []

# категориальные
cat_cols = []

**bodyType**

In [ ]:
data.bodyType.value_counts()

In [ ]:
data['bodyType'] = data['bodyType'].astype(str).apply(lambda x: None if x.strip()=='' else x)
data['bodyType'] = data.bodyType.apply(lambda x: x.split(' ')[0].lower())

In [ ]:
data.bodyType.value_counts()

In [ ]:
#посмотрим на распределение
data.bodyType.value_counts().plot.barh() 

In [ ]:
cat_cols.append('bodyType')

**brand**

In [ ]:
data.brand.value_counts()

In [ ]:
cat_cols.append('brand')

**color**

In [ ]:
data.color.value_counts()

In [ ]:
data.color.value_counts().plot.barh() 

In [ ]:
cat_cols.append('color')

**Description**

In [ ]:
# description - описание, комментарий продавца
# сгенерируем из него новый числовой признак
data['description_len'] = data.description.apply(lambda x: len(str(x)))

In [ ]:
data = data.drop('description',axis=1)

In [ ]:
num_cols.append('description_len')

**fuelType**

In [ ]:
data.fuelType.value_counts()

In [ ]:
sns.countplot(x = 'fuelType', data = data)

In [ ]:
cat_cols.append('fuelType')

**image**

In [ ]:
# проверим фото на уникальность
count_photos = data.image.value_counts()
display(count_photos.head(5))

In [ ]:
#получим уникальные фотографии
display(count_photos.sort_values().head(5))

In [ ]:
# сделаем из него бинарный признак по типу уникальности фото
data['real_photo'] = data.image.apply(lambda x: x.find('get-verba') == -1)

real = data[data['real_photo'] == False].image.value_counts()
display(real.sort_values().head(5))

In [ ]:
data = data.drop('image',axis=1)

In [ ]:
data['real_photo'].value_counts()

In [ ]:
sns.countplot(x = 'real_photo', data = data)

In [ ]:
bin_cols.append('real_photo')

**engineDisplacement**

In [ ]:
data.engineDisplacement.value_counts()

In [ ]:
#оставим только численное значение 
data.engineDisplacement = data.engineDisplacement.apply(lambda x: x[:3])
data.engineDisplacement = data.engineDisplacement.replace(' LT', '2.0')
data.engineDisplacement = data.engineDisplacement.apply(lambda x: float(x))

In [ ]:
#разобьем по категориям
data["engineDisplacement"] = data["engineDisplacement"].astype('float').apply(lambda x: 4 if x >= 4 else 
   3 if x >= 3 else(2 if x >= 2 else (1 if x >= 1 else 0)))
data.engineDisplacement.value_counts()

In [ ]:
sns.countplot(x = 'engineDisplacement', data = data)

In [ ]:
cat_cols.append('engineDisplacement')

**enginePower**

In [ ]:
data.enginePower.value_counts()

In [ ]:
#преобразуем данные
data['enginePower'] = data['enginePower'].apply(lambda x: x[:3])
data['enginePower'] = data['enginePower'].replace('7 N','7')
data['enginePower'] = data['enginePower'].apply(lambda x: int(x))

In [ ]:
data.enginePower.hist().barh

In [ ]:
num_cols.append('enginePower')

**mileage**

In [ ]:
data['mileage'].value_counts()

In [ ]:
#Очистка
data['mileage'] = data['mileage'].apply(lambda x: str(x).replace('\xa0км', ''))

In [ ]:
data['mileage'] = data.mileage.apply(
    lambda x: int("".join(filter(str.isdigit, x))))

In [ ]:
data['mileage'].hist(figsize=(5,5), bins=50)
data.mileage.describe()

In [ ]:
#Создадим признак 'mileage_cat'  
data["mileage_cat"] = data["mileage"].apply(lambda x: 4 if x>=40000 else(3 if x>=30000 else(2 if x>=20000 else (1 if x>=10000 else 0))))

In [ ]:
data.mileage_cat.value_counts()

In [ ]:
num_cols.append('mileage')
cat_cols.append('mileage_cat')

**productionDate и modelDate**

In [ ]:
data.productionDate.value_counts()

In [ ]:
print(data['productionDate'].hist(figsize=(5,5), bins=50))

In [ ]:
data.modelDate.value_counts()

In [ ]:
print(data['modelDate'].hist(figsize=(5,5), bins=50))

In [ ]:
#создадим новые признаки на основе существующих: количество лет производства и возраст модели
data['prod_age'] = 2021 - data.productionDate
data['model_age'] = 2021 - data.modelDate

In [ ]:
#определим эти признаки в числовые данные
num_cols.append('productionDate')
num_cols.append('modelDate')
num_cols.append('prod_age')
num_cols.append('model_age')

**model_name**

In [ ]:
data.model_name.unique()

In [ ]:
cat_cols.append('model_name')

**numberOfDoors**

In [ ]:
data.numberOfDoors.value_counts() 

In [ ]:
sns.countplot(x = 'numberOfDoors', data = data)

In [ ]:
cat_cols.append('numberOfDoors')

**sell_id**

In [ ]:
test.sell_id.isna().sum()

sell_id - идентификатор объявления,который у нас есть в test-вборке. Пропусков не осталось. Смысловой нагрузки не несет, для обучение не берем.

**vehicleTransmission**

In [ ]:
data.vehicleTransmission.value_counts()

In [ ]:
sns.countplot(x = 'vehicleTransmission', data = data)

In [ ]:
#определяем в категориальный признак
cat_cols.append('vehicleTransmission')

**Владельцы**

In [ ]:
data['Владельцы'].value_counts()

In [ ]:
#Очистка
data['Владельцы'] = data['Владельцы'].apply(lambda x: int(x[0])).astype('int32')

In [ ]:
data.Владельцы.value_counts()

In [ ]:
sns.countplot(x = 'Владельцы', data = data)

In [ ]:
cat_cols.append('Владельцы')

**ПТС**

In [ ]:
data.ПТС.value_counts()

In [ ]:
data.ПТС.isna().sum()

In [ ]:
data.ПТС = data.ПТС.fillna('Оригинал')# заполним пропуск наиболее частым значением

In [ ]:
# Преобразуем значения в колонке ПТС
data['ПТС'] = data['ПТС'].apply(lambda x: 1 if x == 'Оригинал' else 0).astype('int8')

In [ ]:
sns.countplot(x = 'ПТС', data = data)

In [ ]:
#определяем в бинарные признаки
bin_cols.append('ПТС')

**Привод**

In [ ]:
data.Привод.value_counts()

In [ ]:
#посмотрим на распределение
sns.countplot(x = 'Привод', data = data)

In [ ]:
cat_cols.append('Привод')

**Руль**

In [ ]:
data.Руль.value_counts()

In [ ]:
# Преобразуем значения в колонке Руль
data['Руль'] = data['Руль'].apply(lambda x: 1 if x == 'Левый' else 0).astype('int8')

In [ ]:
sns.countplot(x = 'Руль', data = data)

In [ ]:
bin_cols.append('Руль')

**Состояние**

In [ ]:
data.Состояние.unique()

In [ ]:
# Преобразуем значения в колонке Состояние
data['Состояние'] = data['Состояние'].apply(lambda x: 1 if x == 'Не требует ремонта' else 0).astype('int8')

In [ ]:
data.Состояние.value_counts()

In [ ]:
bin_cols.append('Состояние')

**Посмотрим на распределение целевой переменной:**

In [ ]:
# приведем целевую переменную к числовому типу
data['price'] = data['price'].apply(lambda x: str(x))

In [ ]:
data.price = data.price.apply(
    lambda x: ("".join(filter(str.isdigit, x)))).astype('int')

In [ ]:
plt.figure()
plt.title(f"Распределение {'price'}")
sns.distplot(data.price, kde=False)

Посмотрим на распределение зависимости цены от марки авто:

In [ ]:
plt.figure(figsize=(10, 15))
plt.scatter(np.log(data.price), data.brand)

Показывает что цены адекватны и зависят от марки авто

In [ ]:
#Посмотрим на распределение зависимости цены от километража
plt.figure(figsize=(10, 15))
plt.scatter(np.log(data.price), data.mileage)

 Удалим признаки, которые на наш взгляд не несут никакой информативности,либо дублируют информацию:

In [ ]:
data = data.drop(columns = ['car_url','name','sell_id','parsing_unixtime','vehicleConfiguration','priceCurrency','Таможня'], axis=1)

In [ ]:
data.head(5)

## Label Encoding

In [ ]:
for colum in ['bodyType','color','model_name','brand','fuelType', 'vehicleTransmission','Привод','real_photo']:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data.sample(5)

In [ ]:
print('Категориальныe: {} \nБинарные: {}\nЧисловые: {}'.format(cat_cols, bin_cols,num_cols))

# Бинарные признаки

In [ ]:
for i in bin_cols:
    display(data[i].value_counts())

In [ ]:
#Проверим значимость бинарных переменных:
df = data[data['sample'] == 0]
imp_cat = Series(mutual_info_classif(data[bin_cols], data['price'],
                                     discrete_features =True), index = bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

по графику видим,что сильно больше всего влияют признаки ПТС и Руль на цену

In [ ]:
# построим матрицу корреляций для числовых признаков
plt.figure(figsize=(10,5))
sns.heatmap(data[num_cols+['price']].corr(), annot=True)

# Категориальные признаки

In [ ]:
# Проверим значимость категориальных переменных:
temp_df = data[data['sample']==1]
imp_cat = Series(mutual_info_classif(data[cat_cols], data['price'],
                                     discrete_features=True), index=cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

Логично,что цена будет зависеть от марки и модели машины

# Числовые признаки

In [ ]:
# анализ распределения числовых переменных с помощью визуализации
for col in num_cols:
    plt.figure()
    plt.title(f"Распределение {col}")
    sns.distplot(data[col], kde=False)

In [ ]:
imp_num = Series(f_classif(data[num_cols][data['sample'] == 1], data[data['sample'] == 1]['price'])[0], 
                 index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Все признаки достаточно сильно влияют на цену

In [ ]:
# корреляция

fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax = sns.heatmap(data.corr(),fmt='.1g',
                 annot=True, cmap='coolwarm')

Видна большая корреляция между признаками 'engineDisplacement' и 'enginePower',а так же искуственно созданными признаками. Удалять не буду, т.к. без них результат становится хуже.

## Train Split

In [ ]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample','price'], axis=1)
X = train_data.drop(['price'], axis=1)
y = train_data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 



In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())
#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# Простая модель линейной регрессии

In [ ]:
from sklearn.linear_model import LinearRegression
linear_regr = LinearRegression().fit(X_train, np.log(y_train+1))
predict_test = np.exp(linear_regr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# CatBoost


In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

### Log Target
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели.    

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')
predict_test = np.exp(model.predict(X_test))
#predict_submission = np.exp(model.predict(test_data))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# xgboost

In [ ]:
# xgboost
import xgboost as xgb
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")
VERSION = 7
predict_test = np.exp(xb.predict(X_test))
predict_submission = np.exp(xb.predict(test_data))

# GradientBoosting с подбором параметров

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
def regularise(X_train, y_train):
    max_depth = [5, 10,15]
    n_estimators = [100, 200,1000]
    hyperparameters = dict(max_depth=max_depth, n_estimators=n_estimators)
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)

    clf = GridSearchCV(model, hyperparameters)

    best_model = clf.fit(X_train, y_train)

    best_max_depth = best_model.best_estimator_.get_params()['max_depth']
    best_n_estimators = best_model.best_estimator_.get_params()['n_estimators']

    return best_max_depth, best_n_estimators

In [ ]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=5, n_estimators=1000)
gb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(gb.predict(X_test))))*100:0.2f}%")

# Stacking

In [ ]:
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 


y_train = y_train 
y_test = y_test

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

def compute_meta_feature(regr, X_train, X_test, y_train, cv):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    
    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)

    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)

    X_meta_test = meta_regr.predict(X_test)


    return X_meta_train, X_meta_test

def generate_meta_features(regr, X_train, X_test, y_train, cv):
    features = [compute_meta_feature(regr, X_train, X_test, y_train, cv) for regr in tqdm(regr)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test in features]).T
    return stacked_features_train, stacked_features_test

X_train = np.where(np.isnan(X_train), 0, X_train)
X_test = np.where(np.isnan(X_test), 0, X_test)
y_train = np.where(np.isnan(y_train), 0, y_train)

In [ ]:
regr = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                             max_features=3, max_depth=19, bootstrap=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test = generate_meta_features([
                            regr,
                            GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300),
                            KNeighborsRegressor(n_neighbors=2, algorithm = 'ball_tree', weights = 'distance', p=1),
                            RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1, max_depth=5, n_estimators=200),
                            ExtraTreesRegressor(random_state=RANDOM_SEED), 
                            RandomForestRegressor(random_state=RANDOM_SEED, max_depth=15) \
], X_train, X_test, y_train, cv)

In [ ]:
def compute_metric(regr, X_train, y_train, X_test, y_test): 
    regr.fit(X_train, y_train) 
    y_test_pred = regr.predict(X_test) 
    return np.round(mape(y_test, y_test_pred)*100, 2)
print(f"Точность модели по метрике MAPE: {compute_metric(regr, stacked_features_train, y_train, stacked_features_test, y_test)}%")

# Выводы

* CatBoost и GradientBoosting показывают хороший результат метрики, но лучший результат показал xgboost - 12.13.
* Заметно улучшается результат после логарифмирования целевой переменной


# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)